In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="6"
print(os.environ["CUDA_VISIBLE_DEVICES"])

6


In [10]:
from PIL import Image
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

import lavis
from lavis.models import load_model_and_preprocess

import random 
random.seed(43)

EXT = ['.jpg', '.jpeg', '.png']

class TextInvDataset(Dataset):
    def __init__(self, roots, labels, vis_processors=None, txt_processors=None):
        
        self.path_and_labels = {
            'img_path': [],
            'label': []
        }
        
        assert len(roots) == len(labels), "Please assign a label for each image root."
        
        for root, label in zip(roots, labels):
            n_sample = 0
            for r, dirs, files in os.walk(root):
                for file in files:
                    if os.path.splitext(file)[-1] in EXT:
                        self.path_and_labels["img_path"].append(os.path.join(r, file))
                        self.path_and_labels["label"].append(label)
                        n_sample += 1
            print(f'Found {n_sample} images with label "{label}".')
        
        self.path_and_labels = pd.DataFrame.from_dict(self.path_and_labels)
        self.path_and_labels.set_index("img_path", inplace=True)
        
        self.vis_processors = vis_processors
        self.txt_processors = txt_processors

    def __len__(self):
        
        return len(list(self.path_and_labels.index))

    def __getitem__(self, index):

        image_path = list(self.path_and_labels.index)[index]
        image = Image.open(image_path).convert("RGB")
        if self.vis_processors:
            image = self.vis_processors(image)
        
        label = self.path_and_labels.loc[image_path, "label"]

        return image, label

# This is for query lots of images

from os import listdir
from os.path import isfile, join
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix
import time

class InstructBLIP():
    def __init__(self, name="blip2_vicuna_instruct_textinv", model_type="vicuna7b", is_eval=True, device="cpu") -> None:
        print(f'Loading model...')
        #self.model, self.vis_processors, self.txt_processors = load_model_and_preprocess(name, model_type, is_eval, device)
        self.imgs = []
        self.labels = []
        
        # QA
        self.question = ""
        
        # results
        self.acc = None
        self.confusion_mat = None
        
        self.acc_3class = None
        self.confusion_mat_3class = None
        
        self.com_acc = None
        self.com_confusion_mat = None
        self.uncom_acc = None
        self.uncom_confusion_mat = None

    def LoadModels(self, model, vis_processors, txt_processors, device):
        self.model = model
        self.vis_processors = vis_processors
        self.txt_processors = txt_processors
        self.device = device
        
    def LoadData(self, roots, labels):
        self.roots = [roots] if isinstance(roots, str) else roots
        self.text_labels = [labels] if isinstance(labels, str) else labels
        self.dataset = TextInvDataset(self.roots, self.text_labels, vis_processors=self.vis_processors["eval"])
        self.dataloader = DataLoader(dataset=self.dataset, batch_size=8, shuffle=False, num_workers=8)    
    
    def QueryImgs_batch(self, question, true_string="yes", logPath='log.txt'):
        self.labels = []
        self.label_3class = []
        self.ans_list = []
        self.question = question
        
        for image, label in tqdm(self.dataloader):
            
            image = image.to(self.device)
            
            questions = [self.question] * image.shape[0]
            
            # samples = {"image": image, "text_input": questions}
            # ans = self.model.predict_answers(samples=samples, inference_method="generate", answer_list=["yes", "no"])
            # pred_label = [0 if a == true_string else 1 for a in ans]
            
            samples = {"image": image, "prompt": questions}
            candidates = ["yes", "no"]
            ans = self.model.predict_class(samples=samples, candidates=candidates)
            pred_label = [0 if candidates[list(a).index(0)]==true_string else 1 for a in ans]
            self.ans_list += pred_label
            
            label = [0 if l == true_string else 1 for l in label]
            self.labels += label
        
        self.acc = accuracy_score(self.labels, self.ans_list)
        self.confusion_mat = confusion_matrix(self.labels, self.ans_list, labels=[0,1])
        
        self.ans_list = np.array(self.ans_list)
        self.labels = np.array(self.labels)
        self.label_3class = np.array(self.label_3class)
        
        self.PrintResult(detailed=True, logPath=logPath)
        
        return self.acc, self.confusion_mat, self.ans_list, self.labels, self.label_3class
    
    def Query(self, image, question):
        image = self.vis_processors["eval"](image).unsqueeze(0).to(self.device)
        
        samples = {"image": image, "prompt": question}
        candidates = ["yes", "no"]
        ans = self.model.predict_class(samples=samples, candidates=candidates)
        pred_label = ["True" if candidates[list(a).index(0)]=="yes" else "Fake" for a in ans]
        return pred_label

    def PrintResult(self, detailed=False, acc=None, confusion_mat=None, ans_list=None, labels=None, logPath=None):
        
        if acc:
            self.acc = acc
        if confusion_mat:
            self.confusion_mat = confusion_mat
        if ans_list:
            self.ans_list = ans_list
        if labels:
            self.labels = labels
        
        if logPath:
            logfile = open(logPath, 'a')
        
        if detailed:
            
            print(f'[TIME]      : {time.ctime()}', file=logfile)
            print(f'[Finetuned] : {self.model.finetuned}', file=logfile)
            print(f'[Img roots] : {self.roots}', file=logfile)
            print(f'[Labels]    : {self.text_labels}', file=logfile)
            print(f'[Question]  : {self.question}\n', file=logfile)
            
            print(f'=== Overall ===', file=logfile)
            print(f'Acc: {self.acc*100:.2f}%', file=logfile)
            self.PrintConfusion(self.confusion_mat, logfile=logfile)
            print('\n', file=logfile)
            
            if 0 in self.labels:
                real_ans_list = self.ans_list[self.labels==0]
                real_label = [0] * len(real_ans_list)
                self.real_acc = accuracy_score(real_label, real_ans_list)
                self.real_confusion_mat = confusion_matrix(real_label, real_ans_list, labels=[0,1])
                print(f'=== Real images ===', file=logfile)
                print(f'Acc: {self.real_acc*100:.2f}%', file=logfile)
                self.PrintConfusion(self.real_confusion_mat, logfile=logfile)
                print('\n', file=logfile)
            else:
                print(f'=== No real images ===\n', file=logfile)
            
            
            if 1 in self.labels:
                fake_ans_list = self.ans_list[self.labels==1]
                fake_label = [1] * len(fake_ans_list)
                self.com_acc = accuracy_score(fake_label, fake_ans_list)
                self.com_confusion_mat = confusion_matrix(fake_label, fake_ans_list, labels=[0,1])
                print(f'=== Fake images ===', file=logfile)
                print(f'Acc: {self.com_acc*100:.2f}%', file=logfile)
                self.PrintConfusion(self.com_confusion_mat, logfile=logfile)
                print('\n', file=logfile)
            else:
                print(f'=== No fake images ===\n', file=logfile)
        else:
            print(f'Question: {self.question}\n', file=logfile)
            print(f'Acc: {self.acc*100:.2f}%', file=logfile)
            self.PrintConfusion(self.confusion_mat, logfile=logfile)
            print('\n', file=logfile)
        
        logfile.close()
    
    def PrintConfusion(self, mat, logfile):
        padding = ' '
        print(f'        | Pred real | Pred fake |', file=logfile)
        print(f'GT real | {mat[0, 0]:{padding}<{10}}| {mat[0, 1]:{padding}<{11}}|', file=logfile)
        print(f'GT fake | {mat[1, 0]:{padding}<{10}}| {mat[1, 1]:{padding}<{11}}|', file=logfile)
        
    def MultipleAns(self, ans1, ans2):
    
        # Q1: Is this photo common in real world?
        # Q2: Is this photo generated by a model?
        
        final_ans = []
        for ans in zip(ans1, ans2):
            if ans[0] == 0 and ans[1] == 0:
                final_ans.append(0)
            else:
                final_ans.append(1)
        
        acc = accuracy_score(self.labels, final_ans)
        confusion_mat = confusion_matrix(self.labels, final_ans)
        print(f'Accuracy: {acc*100:.2f}%')
        self.PrintConfusion(confusion_mat)
        
        self.ans_list = final_ans
        self.acc = acc
        self.confusion_mat = confusion_mat
        
        return acc, confusion_mat, final_ans
    
def print_combine_result(pretrained_ans, finetuned_ans, label, logPath):
    
    logfile = open(logPath, 'a')
    
    def _print_confusion(mat, logfile):
        padding = ' '
        print(f'        | Pred real | Pred fake |', file=logfile)
        print(f'GT real | {mat[0, 0]:{padding}<{10}}| {mat[0, 1]:{padding}<{11}}|', file=logfile)
        print(f'GT fake | {mat[1, 0]:{padding}<{10}}| {mat[1, 1]:{padding}<{11}}|', file=logfile)
    
    comb_ans = np.ceil((pretrained_ans + finetuned_ans)/2).astype(np.int64)
    
    comb_acc = accuracy_score(label, comb_ans)
    comb_confusion_mat = confusion_matrix(label, comb_ans, labels=[0,1])
    
    print(f'=== Overall (Comb) ===', file=logfile)
    print(f'Acc: {comb_acc*100:.2f}%', file=logfile)
    _print_confusion(comb_confusion_mat, logfile=logfile)
    print('\n', file=logfile)
    
    real_ans_list = comb_ans[label==0]
    real_label = [0] * len(real_ans_list)
    real_acc = accuracy_score(real_label, real_ans_list)
    real_confusion_mat = confusion_matrix(real_label, real_ans_list, labels=[0,1])
    print(f'=== Real images (Comb) ===', file=logfile)
    print(f'Acc: {real_acc*100:.2f}%', file=logfile)
    _print_confusion(real_confusion_mat, logfile=logfile)
    print('\n', file=logfile)
    
    
    com_ans_list = comb_ans[label==1]
    com_label = [1] * len(com_ans_list)
    com_acc = accuracy_score(com_label, com_ans_list)
    com_confusion_mat = confusion_matrix(com_label, com_ans_list, labels=[0,1])
    print(f'=== Common fake images (Comb) ===', file=logfile)
    print(f'Acc: {com_acc*100:.2f}%', file=logfile)
    _print_confusion(com_confusion_mat, logfile=logfile)
    print('\n', file=logfile)
    
    return comb_acc, comb_confusion_mat, comb_ans

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_vicuna_instruct_textinv", model_type="vicuna7b", is_eval=True, device=device)

print(f'Load model OK!')

tokenizer OK!
visual encoder OK!
Q-former OK!


Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.54s/it]


LLM OK!
Added 1 tokens to Qformer tokenizer.
Added 1 tokens to llm tokenizer.
Missing keys ['visual_encoder.cls_token', 'visual_encoder.pos_embed', 'visual_encoder.patch_embed.proj.weight', 'visual_encoder.patch_embed.proj.bias', 'visual_encoder.blocks.0.norm1.weight', 'visual_encoder.blocks.0.norm1.bias', 'visual_encoder.blocks.0.attn.q_bias', 'visual_encoder.blocks.0.attn.v_bias', 'visual_encoder.blocks.0.attn.qkv.weight', 'visual_encoder.blocks.0.attn.proj.weight', 'visual_encoder.blocks.0.attn.proj.bias', 'visual_encoder.blocks.0.norm2.weight', 'visual_encoder.blocks.0.norm2.bias', 'visual_encoder.blocks.0.mlp.fc1.weight', 'visual_encoder.blocks.0.mlp.fc1.bias', 'visual_encoder.blocks.0.mlp.fc2.weight', 'visual_encoder.blocks.0.mlp.fc2.bias', 'visual_encoder.blocks.1.norm1.weight', 'visual_encoder.blocks.1.norm1.bias', 'visual_encoder.blocks.1.attn.q_bias', 'visual_encoder.blocks.1.attn.v_bias', 'visual_encoder.blocks.1.attn.qkv.weight', 'visual_encoder.blocks.1.attn.proj.weight', 

In [18]:
logPath = '/home/denny/LAVIS/deepfake-detection/log/log.txt'
# logPath = '/home/denny/LAVIS/deepfake-detection/log/log2.txt'
# logPath = '/home/denny/LAVIS/deepfake-detection/log/SD2_SD2IP_90k_lora_onlyCommon.txt'
# logPath = '/home/denny/LAVIS/deepfake-detection/log/SD2_SD2IP_90k_lora_onlyCommon2.txt'

q1 = "Is this photo real?"
q2 = "Is this photo real [*]?"

file = open(logPath, 'a')
file.close()

instruct = InstructBLIP()
instruct.LoadModels(model, vis_processors, txt_processors, device)

print(f'Log path: {logPath}')
print(f'Q1: {q1}')
# print(f'Q2: {q2}')

# csvfiles = [
# # "/eva_data0/denny/textual_inversion/debug_label.csv",
# "/eva_data0/denny/textual_inversion/60k_6k_6k/test_COCO_label.csv",
# # "/eva_data0/denny/textual_inversion/60k_6k_6k/test_Flickr_label.csv",
# # "/eva_data0/denny/textual_inversion/60k_6k_6k/test_SD2_label.csv",
# # "/eva_data0/denny/textual_inversion/60k_6k_6k/test_SDXL_label.csv", 
# # "/eva_data0/denny/textual_inversion/60k_6k_6k/test_IF_label.csv",
# # "/eva_data0/denny/textual_inversion/60k_6k_6k/test_DALLE_label.csv",
# # "/eva_data0/denny/textual_inversion/60k_6k_6k/test_SGXL_label.csv",
# # "/eva_data0/denny/textual_inversion/60k_6k_6k/test_Control_COCO_label.csv",
# # "/eva_data0/iammingggg/textual_inversion/60k_6k_6k/test_lama_label.csv",
# # "/eva_data0/iammingggg/textual_inversion/60k_6k_6k/test_SD2IP_label.csv",
# # "/eva_data0/iammingggg/textual_inversion/60k_6k_6k/test_lte_label.csv",
# # "/eva_data0/iammingggg/textual_inversion/60k_6k_6k/test_SD2SR_label.csv",
# # "/eva_data0/iammingggg/textual_inversion/60k_6k_6k/test_deeperforensics_faceOnly_label.csv",
# # "/eva_data0/denny/textual_inversion/60k_6k_6k/test_AdvAtk_Imagenet_label.csv",
# # "/eva_data0/denny/textual_inversion/60k_6k_6k/test_Backdoor_Imagenet_label.csv",
# # "/eva_data0/denny/textual_inversion/60k_6k_6k/test_DataPoison_Imagenet_label.csv",
# ]

roots_and_labels = [
    ["/eva_data0/denny/textual_inversion/debug/0_real/", "yes"],
    ["/eva_data0/denny/textual_inversion/debug/0_real/", "yes"],
]

for root, label in roots_and_labels:
    instruct.LoadData(roots=root, labels=label)

    question = q1
    acc, confusion_mat, pretrained_ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)
    print(f'Question: {question}')
    print(f'     Acc: {acc*100:.2f}%\n')

    # question = q2
    # acc, confusion_mat, finetuned_ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)
    # print(f'Question: {question}')
    # print(f'Acc: {acc*100:.2f}%')

    # comb_acc, comb_confusion_mat, comb_ans = print_combine_result(pretrained_ans_list, finetuned_ans_list, labels, logPath=logPath)
    # print(f'[Combination]')
    # print(f'Acc: {comb_acc*100:.2f}%')

Loading model...
Log path: /home/denny/LAVIS/deepfake-detection/log/log.txt
Q1: Is this photo real?
Found 100 images with label "yes".


  0%|          | 0/13 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:04<00:00,  2.74it/s]


Question: Is this photo real?
     Acc: 98.00%

Found 100 images with label "yes".


100%|██████████| 13/13 [00:04<00:00,  2.77it/s]

Question: Is this photo real?
     Acc: 98.00%



In [14]:
path = "/eva_data0/denny/textual_inversion/debug/1_fake/common/00009.png"
image = Image.open(path)
ans = instruct.Query(image, q1)
print(ans)

['Fake']


In [3]:
class TextInvDataset(Dataset):
    def __init__(self, csv, vis_processors=None, txt_processors=None):
        
        self.path_and_labels = pd.read_csv(csv, index_col="img_path")
        self.vis_processors = vis_processors
        self.txt_processors = txt_processors

    def __len__(self):
        
        return len(list(self.path_and_labels.index))

    def __getitem__(self, index):

        image_path = list(self.path_and_labels.index)[index]
        image = Image.open(image_path).convert("RGB")
        if self.vis_processors:
            image = self.vis_processors(image)
        
        label = self.path_and_labels.loc[image_path, "label"]
        
        is_uncommon = "uncommon" in image_path

        return image, label, is_uncommon

#### Query batch of images

In [4]:

# This is for query lost of images

from os import listdir
from os.path import isfile, join
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix
import time

class InstructBLIP():
    def __init__(self, name="blip2_vicuna_instruct_textinv", model_type="vicuna7b", is_eval=True, device="cpu") -> None:
        print(f'Loading model...')
        #self.model, self.vis_processors, self.txt_processors = load_model_and_preprocess(name, model_type, is_eval, device)
        self.imgs = []
        self.labels = []
        
        # QA
        self.question = ""
        
        # results
        self.acc = None
        self.confusion_mat = None
        
        self.acc_3class = None
        self.confusion_mat_3class = None
        
        self.com_acc = None
        self.com_confusion_mat = None
        self.uncom_acc = None
        self.uncom_confusion_mat = None

    def LoadModels(self, model, vis_processors, txt_processors, device):
        self.model = model
        self.vis_processors = vis_processors
        self.txt_processors = txt_processors
        self.device = device
    
    def LoadImages(self, dir, num):
        onlyfiles = []
        
        for f in sorted(listdir(dir)):
            if isfile(join(dir, f)):
                onlyfiles.append(join(dir, f))
        
        onlyfiles = random.sample(onlyfiles, num)
        
        raw_img_list = []
        with tqdm(total=len(onlyfiles), desc=f'Loading imgs from {dir}') as pbar:
            for f in onlyfiles:
                raw_img = Image.open(f).convert("RGB")
                raw_img_list.append(raw_img)
                pbar.update(1)
        
        return raw_img_list

    def LoadData(self, real_dir, fake_dir, num=1000):
        #real_imgs = LoadImages(join(root_dir, "0_real"))
        #fake_imgs = LoadImages(join(root_dir, "1_fake"))
        real_imgs = self.LoadImages(real_dir, num)
        fake_imgs = self.LoadImages(fake_dir, num)
        
        self.imgs = real_imgs + fake_imgs
        self.labels = [0]*len(real_imgs) + [1]*len(fake_imgs)
        #return self.imgs, self.labels
      
    def LoadData_batch(self, csv_path):
        self.csv = csv_path
        self.dataset = TextInvDataset(csv=csv_path, vis_processors=self.vis_processors["eval"])
        self.dataloader = DataLoader(dataset=self.dataset, batch_size=8, shuffle=False, num_workers=8)    
        
    def LoadData3Class(self, real_dir, fake_common_dir, fake_uncommon_dir, num=[1000, 500, 500]):
        #real_imgs = LoadImages(join(root_dir, "0_real"))
        #fake_imgs = LoadImages(join(root_dir, "1_fake"))
        self.num = num
        real_imgs = self.LoadImages(real_dir, num[0])
        fake_common_imgs = self.LoadImages(fake_common_dir, num[1])
        fake_uncommon_imgs = self.LoadImages(fake_uncommon_dir, num[2])
        
        self.imgs = real_imgs + fake_common_imgs + fake_uncommon_imgs
        self.labels = [0]*len(real_imgs) + [1]*(len(fake_common_imgs)+len(fake_uncommon_imgs))
        self.label_3class = [0]*len(real_imgs) + [1]*len(fake_common_imgs) + [2]*len(fake_uncommon_imgs)
        #return self.imgs, self.labels, self.label_3class

    def QueryImgs(self, question, true_string="yes"):
        self.ans_list = []
        self.question = question
        
        with tqdm(total=len(self.imgs), desc=f'Answering') as pbar:
            for idx, img in enumerate(self.imgs):
                image = self.vis_processors["eval"](img).unsqueeze(0).to(self.device)

                samples = {"image": image, "text_input": question}
                
                ans = self.model.predict_answers(samples=samples, inference_method="generate")[0]
                self.ans_list.append(0 if ans == true_string else 1)
                
                pbar.update(1)
        
        self.acc = accuracy_score(self.labels, self.ans_list)
        self.confusion_mat = confusion_matrix(self.labels, self.ans_list)
        
        self.PrintResult()
        
        return self.acc, self.confusion_mat, self.ans_list
    
    def QueryImgs_batch(self, question, true_string="yes", logPath='log.txt'):
        self.labels = []
        self.label_3class = []
        self.ans_list = []
        self.question = question
        
        for image, label, is_uncommon in tqdm(self.dataloader):
            
            image = image.to(self.device)
            
            questions = [self.question] * image.shape[0]
            samples = {"image": image, "text_input": questions}

            ans = self.model.predict_answers(samples=samples, inference_method="generate")
            pred_label = [0 if a == true_string else 1 for a in ans]
            self.ans_list += pred_label
            
            label = [0 if l == true_string else 1 for l in label]
            self.labels += label
            
            label_3class = label.copy()
            label_3class = [2 if is_uncommon[idx] else l for idx, l in enumerate(label)]
            
            self.label_3class += label_3class
        
        self.acc = accuracy_score(self.labels, self.ans_list)
        self.confusion_mat = confusion_matrix(self.labels, self.ans_list)
        
        self.ans_list = np.array(self.ans_list)
        self.labels = np.array(self.labels)
        self.label_3class = np.array(self.label_3class)
        
        self.PrintResult(three_class=True, logPath=logPath)
        
        return self.acc, self.confusion_mat, self.ans_list, self.labels, self.label_3class
    
    def Query(self, image, question):
        image = self.vis_processors["eval"](image).unsqueeze(0).to(self.device)
        
        samples = {"image": image, "text_input": question}
        ans = self.model.predict_answers(samples=samples, inference_method="generate")[0]
        return ans

    def PrintResult(self, three_class=False, acc=None, confusion_mat=None, ans_list=None, labels=None, label_3class=None, logPath=None):
        
        if acc:
            self.acc = acc
        if confusion_mat:
            self.confusion_mat = confusion_mat
        if ans_list:
            self.ans_list = ans_list
        if labels:
            self.labels = labels
        if label_3class:
            self.label_3class = label_3class
        
        if logPath:
            logfile = open(logPath, 'a')
        
        if three_class:
            #assert type(self.num) == list, "Type of num should be list."
            
            print(f'[TIME]      : {time.ctime()}', file=logfile)
            print(f'[Finetuned] : {self.model.finetuned}', file=logfile)
            print(f'[Data csv]  : {self.csv}', file=logfile)
            print(f'[Question]  : {self.question}\n', file=logfile)
            
            print(f'=== Overall ===', file=logfile)
            print(f'Acc: {self.acc*100:.2f}%', file=logfile)
            self.PrintConfusion(self.confusion_mat, logfile=logfile)
            print('\n', file=logfile)
            
            real_ans_list = self.ans_list[self.label_3class==0]
            real_label = [0] * len(real_ans_list)
            self.real_acc = accuracy_score(real_label, real_ans_list)
            self.real_confusion_mat = confusion_matrix(real_label, real_ans_list, labels=[0,1])
            print(f'=== Real images ===', file=logfile)
            print(f'Acc: {self.real_acc*100:.2f}%', file=logfile)
            self.PrintConfusion(self.real_confusion_mat, logfile=logfile)
            print('\n', file=logfile)
            
            com_ans_list = self.ans_list[self.label_3class==1]
            com_label = [1] * len(com_ans_list)
            self.com_acc = accuracy_score(com_label, com_ans_list)
            self.com_confusion_mat = confusion_matrix(com_label, com_ans_list, labels=[0,1])
            print(f'=== Common fake images ===', file=logfile)
            print(f'Acc: {self.com_acc*100:.2f}%', file=logfile)
            self.PrintConfusion(self.com_confusion_mat, logfile=logfile)
            print('\n', file=logfile)
            
            uncom_ans_list = self.ans_list[self.label_3class==2]
            uncom_label = [1] * len(uncom_ans_list)
            self.uncom_acc = accuracy_score(uncom_label, uncom_ans_list)
            self.uncom_confusion_mat = confusion_matrix(uncom_label, uncom_ans_list, labels=[0,1])
            print(f'=== Uncommon fake images ===', file=logfile)
            print(f'Acc: {self.uncom_acc*100:.2f}%', file=logfile)
            self.PrintConfusion(self.uncom_confusion_mat, logfile=logfile)
            print('\n', file=logfile)
        else:
            print(f'Question: {self.question}\n', file=logfile)
            print(f'Acc: {self.acc*100:.2f}%', file=logfile)
            self.PrintConfusion(self.confusion_mat, logfile=logfile)
            print('\n', file=logfile)
        
        logfile.close()
    
    def PrintConfusion(self, mat, logfile):
        padding = ' '
        print(f'        | Pred real | Pred fake |', file=logfile)
        print(f'GT real | {mat[0, 0]:{padding}<{10}}| {mat[0, 1]:{padding}<{11}}|', file=logfile)
        print(f'GT fake | {mat[1, 0]:{padding}<{10}}| {mat[1, 1]:{padding}<{11}}|', file=logfile)
        
    def MultipleAns(self, ans1, ans2):
    
        # Q1: Is this photo common in real world?
        # Q2: Is this photo generated by a model?
        
        final_ans = []
        for ans in zip(ans1, ans2):
            if ans[0] == 0 and ans[1] == 0:
                final_ans.append(0)
            else:
                final_ans.append(1)
        
        acc = accuracy_score(self.labels, final_ans)
        confusion_mat = confusion_matrix(self.labels, final_ans)
        print(f'Accuracy: {acc*100:.2f}%')
        self.PrintConfusion(confusion_mat)
        
        self.ans_list = final_ans
        self.acc = acc
        self.confusion_mat = confusion_mat
        
        return acc, confusion_mat, final_ans

In [ ]:
# logPath = '/home/denny/LAVIS/deepfake-detection/log/log.txt'
# logPath = '/home/denny/LAVIS/deepfake-detection/log/SD2_SD2IP_90k_lora_onlyCommon.txt'
logPath = '/home/denny/LAVIS/deepfake-detection/log/SD2_SD2IP_90k_lora_onlyCommon2.txt'

q1 = "Is this photo real?"
# q1 = "Is this photo fake?"
# q2 = "Is this photo real?"

file = open(logPath, 'a')
file.close()

instruct = InstructBLIP()
instruct.LoadModels(model, vis_processors, txt_processors, device)

print(f'Log path: {logPath}')
print(f'Q1: {q1}')
# print(f'Q2: {q2}')

csvfiles = [
# "/eva_data0/denny/textual_inversion/debug_label.csv",
# "/eva_data0/denny/textual_inversion/60k_6k_6k/test_COCO_label.csv",
# "/eva_data0/denny/textual_inversion/60k_6k_6k/test_Flickr_label.csv",
# "/eva_data0/denny/textual_inversion/60k_6k_6k/test_SD2_label.csv",
# "/eva_data0/denny/textual_inversion/60k_6k_6k/test_SDXL_label.csv", 
# "/eva_data0/denny/textual_inversion/60k_6k_6k/test_IF_label.csv",
# "/eva_data0/denny/textual_inversion/60k_6k_6k/test_DALLE_label.csv",
# "/eva_data0/denny/textual_inversion/60k_6k_6k/test_SGXL_label.csv",
# "/eva_data0/denny/textual_inversion/60k_6k_6k/test_Control_COCO_label.csv",
"/eva_data0/iammingggg/textual_inversion/60k_6k_6k/test_lama_label.csv",
"/eva_data0/iammingggg/textual_inversion/60k_6k_6k/test_SD2IP_label.csv",
"/eva_data0/iammingggg/textual_inversion/60k_6k_6k/test_lte_label.csv",
"/eva_data0/iammingggg/textual_inversion/60k_6k_6k/test_SD2SR_label.csv",
"/eva_data0/iammingggg/textual_inversion/60k_6k_6k/test_deeperforensics_faceOnly_label.csv",
"/eva_data0/denny/textual_inversion/60k_6k_6k/test_AdvAtk_Imagenet_label.csv",
"/eva_data0/denny/textual_inversion/60k_6k_6k/test_Backdoor_Imagenet_label.csv",
"/eva_data0/denny/textual_inversion/60k_6k_6k/test_DataPoison_Imagenet_label.csv",
]

for csv_path in csvfiles:
instruct.LoadData_batch(csv_path=csv_path)
print(f'Load data from {csv_path}')

question = q1
acc, confusion_mat, pretrained_ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)
print(f'Question: {question}')
print(f'Acc: {acc*100:.2f}%')

# question = q2
# acc, confusion_mat, finetuned_ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)
# print(f'Question: {question}')
# print(f'Acc: {acc*100:.2f}%')

# comb_acc, comb_confusion_mat, comb_ans = print_combine_result(pretrained_ans_list, finetuned_ans_list, labels, logPath=logPath)
# print(f'[Combination]')
# print(f'Acc: {comb_acc*100:.2f}%')

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"
# model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_vicuna_instruct", model_type="vicuna7b", is_eval=True, device=device)
model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_vicuna_instruct_textinv", model_type="vicuna7b", is_eval=True, device=device)

tokenizer OK!
visual encoder OK!
Q-former OK!


Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.38s/it]


LLM OK!
Added 1 tokens to Qformer tokenizer.
Added 1 tokens to llm tokenizer.
Qformer.bert.embeddings.train_word_embeddings.weight True
llm_model.model.train_embed_tokens.weight True


In [8]:
instruct = InstructBLIP()
instruct.LoadModels(model, vis_processors, txt_processors, device)

# logPath = 'log/SDXL_postfix_onlyCommon.txt'
logPath = 'log/log.txt'
q1 = "Is this photo real?"
q2 = "Is this photo real [*]?"

Loading model...


In [9]:
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_COCO_SD2_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

question = q1
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

question = q2
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

  0%|          | 0/750 [00:00<?, ?it/s]

  0%|          | 0/750 [00:01<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2 instruct.LoadData_batch(csv_path=csv_path)                                                   │
│   3                                                                                              │
│   4 question = q1                                                                                │
│ ❱ 5 acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=q     │
│   6                                                                                              │
│   7 question = q2                                                                                │
│   8 acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=q     │
│                                                                                                  │
│ in QueryImgs_batch:119                                                                           │
│                                                                                                  │
│   116 │   │   │   samples = {"image": image, "text_input": questions}                            │
│   117 │   │   │   samplesForOut = {"image": image, "text_input": questions, "text_output": lab   │
│   118 │   │   │                                                                                  │
│ ❱ 119 │   │   │   out = self.model(samplesForOut) # for debug                                    │
│   120 │   │   │   ans = self.model.predict_answers(samples=samples, inference_method="generate   │
│   121 │   │   │   pred_label = [0 if a == true_string else 1 for a in ans]                       │
│   122 │   │   │   self.ans_list += pred_label                                                    │
│                                                                                                  │
│ /home/denny/anaconda3/envs/instblip/lib/python3.9/site-packages/torch/nn/modules/module.py:1501  │
│ in _call_impl                                                                                    │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /home/denny/LAVIS/lavis/models/blip2_models/blip2_vicuna_instruct_textinv.py:252 in forward      │
│                                                                                                  │
│   249 │   │                                                                                      │
│   250 │   │   image = samples["image"]                                                           │
│   251 │   │   with self.maybe_autocast():                                                        │
│ ❱ 252 │   │   │   image_embeds = self.ln_vision(self.visual_encoder(image))                      │
│   253 │   │   image_atts = torch.ones(image_embeds.size()[:-1], dtype=torch.long).to(image.dev   │
│   254 │   │                                                

In [8]:
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_COCO_SDXL_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

question = q1
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

question = q2
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

100%|██████████| 750/750 [07:51<00:00,  1.59it/s]
/home/denny/anaconda3/envs/instblip/lib/python3.9/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/denny/anaconda3/envs/instblip/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 750/750 [09:30<00:00,  1.31it/s]
/home/denny/anaconda3/envs/instblip/lib/python3.9/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/denny/anaconda3/envs/instblip/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [9]:
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_COCO_IF_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

question = q1
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

question = q2
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

100%|██████████| 750/750 [07:52<00:00,  1.59it/s]
/home/denny/anaconda3/envs/instblip/lib/python3.9/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/denny/anaconda3/envs/instblip/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 750/750 [09:27<00:00,  1.32it/s]
/home/denny/anaconda3/envs/instblip/lib/python3.9/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/denny/anaconda3/envs/instblip/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_COCO_DALLE_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

question = q1
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

question = q2
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

100%|██████████| 750/750 [09:18<00:00,  1.34it/s]


In [11]:
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_Flickr_SD2_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

question = q1
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

question = q2
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

100%|██████████| 750/750 [09:27<00:00,  1.32it/s]


In [12]:
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_Flickr_SDXL_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

question = q1
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

question = q2
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

100%|██████████| 750/750 [09:31<00:00,  1.31it/s]


In [13]:
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_Flickr_IF_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

question = q1
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

question = q2
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

  0%|          | 0/750 [00:00<?, ?it/s]

100%|██████████| 750/750 [09:34<00:00,  1.30it/s]


In [14]:
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_Flickr_DALLE_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

question = q1
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

question = q2
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes", logPath=logPath)

100%|██████████| 750/750 [09:19<00:00,  1.34it/s]


In [7]:
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_Flickr_IF_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

question = "Is this photo real?"
# question = "Is this photo [*]?"
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes")
instruct.PrintResult(three_class=True)

100%|██████████| 750/750 [08:21<00:00,  1.50it/s]

Question: Is this photo real?

Acc: 70.57%
         | Pred true | Pred false |
GT true  | 2990      | 10         |
GT false | 1756      | 1244       |
Question: Is this photo real?

=== Overall ===
Acc: 70.57%
         | Pred true | Pred false |
GT true  | 2990      | 10         |
GT false | 1756      | 1244       |


=== Real images ===
Acc: 99.67%
         | Pred true | Pred false |
GT true  | 2990      | 10         |
GT false | 0         | 0          |


=== Common fake images ===
Acc: 21.33%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 1180      | 320        |


=== Uncommon fake images ===
Acc: 61.60%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 576       | 924        |


In [8]:
# csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_label.csv"
# csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_COCO_SDXL_label.csv"
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_Flickr_IF_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

# question = "Is this photo real?"
question = "Is this photo [*]?"
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes")
instruct.PrintResult(three_class=True)

100%|██████████| 750/750 [09:25<00:00,  1.33it/s]

Question: Is this photo [*]?

Acc: 95.40%
         | Pred true | Pred false |
GT true  | 2862      | 138        |
GT false | 138       | 2862       |
Question: Is this photo [*]?

=== Overall ===
Acc: 95.40%
         | Pred true | Pred false |
GT true  | 2862      | 138        |
GT false | 138       | 2862       |


=== Real images ===
Acc: 95.40%
         | Pred true | Pred false |
GT true  | 2862      | 138        |
GT false | 0         | 0          |


=== Common fake images ===
Acc: 93.07%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 104       | 1396       |


=== Uncommon fake images ===
Acc: 97.73%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 34        | 1466       |


In [9]:
# csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_label.csv"
# csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_COCO_SDXL_label.csv"
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_COCO_IF_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

question = "Is this photo real?"
# question = "Is this photo [*]?"
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes")
instruct.PrintResult(three_class=True)

100%|██████████| 750/750 [08:32<00:00,  1.46it/s]

Question: Is this photo real?

Acc: 70.20%
         | Pred true | Pred false |
GT true  | 2968      | 32         |
GT false | 1756      | 1244       |
Question: Is this photo real?

=== Overall ===
Acc: 70.20%
         | Pred true | Pred false |
GT true  | 2968      | 32         |
GT false | 1756      | 1244       |


=== Real images ===
Acc: 98.93%
         | Pred true | Pred false |
GT true  | 2968      | 32         |
GT false | 0         | 0          |


=== Common fake images ===
Acc: 21.33%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 1180      | 320        |


=== Uncommon fake images ===
Acc: 61.60%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 576       | 924        |


In [10]:
# csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_label.csv"
# csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_COCO_SDXL_label.csv"
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_COCO_IF_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

# question = "Is this photo real?"
question = "Is this photo [*]?"
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes")
instruct.PrintResult(three_class=True)

100%|██████████| 750/750 [09:30<00:00,  1.31it/s]

Question: Is this photo [*]?

Acc: 95.75%
         | Pred true | Pred false |
GT true  | 2883      | 117        |
GT false | 138       | 2862       |
Question: Is this photo [*]?

=== Overall ===
Acc: 95.75%
         | Pred true | Pred false |
GT true  | 2883      | 117        |
GT false | 138       | 2862       |


=== Real images ===
Acc: 96.10%
         | Pred true | Pred false |
GT true  | 2883      | 117        |
GT false | 0         | 0          |


=== Common fake images ===
Acc: 93.07%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 104       | 1396       |


=== Uncommon fake images ===
Acc: 97.73%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 34        | 1466       |


In [7]:
# csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_label.csv"
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_COCO_SDXL_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

question = "Is this photo real?"
# question = "Is this photo [*]?"
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes")
instruct.PrintResult(three_class=True)

  0%|          | 0/750 [00:00<?, ?it/s]

100%|██████████| 750/750 [08:18<00:00,  1.50it/s]


Question: Is this photo real?

Acc: 67.77%
         | Pred true | Pred false |
GT true  | 2968      | 32         |
GT false | 1902      | 1098       |
Question: Is this photo real?

=== Overall ===
Acc: 67.77%
         | Pred true | Pred false |
GT true  | 2968      | 32         |
GT false | 1902      | 1098       |


=== Real images ===
Acc: 98.93%
         | Pred true | Pred false |
GT true  | 2968      | 32         |
GT false | 0         | 0          |


=== Common fake images ===
Acc: 23.40%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 1149      | 351        |


=== Uncommon fake images ===
Acc: 49.80%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 753       | 747        |


In [8]:
# csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_label.csv"
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_COCO_SDXL_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

# question = "Is this photo real?"
question = "Is this photo [*]?"
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes")
instruct.PrintResult(three_class=True)

  0%|          | 0/750 [00:00<?, ?it/s]

100%|██████████| 750/750 [09:20<00:00,  1.34it/s]

Question: Is this photo [*]?

Acc: 95.58%
         | Pred true | Pred false |
GT true  | 2883      | 117        |
GT false | 148       | 2852       |
Question: Is this photo [*]?

=== Overall ===
Acc: 95.58%
         | Pred true | Pred false |
GT true  | 2883      | 117        |
GT false | 148       | 2852       |


=== Real images ===
Acc: 96.10%
         | Pred true | Pred false |
GT true  | 2883      | 117        |
GT false | 0         | 0          |


=== Common fake images ===
Acc: 93.20%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 102       | 1398       |


=== Uncommon fake images ===
Acc: 96.93%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 46        | 1454       |


In [9]:
# csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_label.csv"
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_Flickr_SDXL_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

question = "Is this photo real?"
# question = "Is this photo [*]?"
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes")
instruct.PrintResult(three_class=True)

  0%|          | 0/750 [00:00<?, ?it/s]

100%|██████████| 750/750 [08:07<00:00,  1.54it/s]

Question: Is this photo real?

Acc: 68.13%
         | Pred true | Pred false |
GT true  | 2990      | 10         |
GT false | 1902      | 1098       |
Question: Is this photo real?

=== Overall ===
Acc: 68.13%
         | Pred true | Pred false |
GT true  | 2990      | 10         |
GT false | 1902      | 1098       |


=== Real images ===
Acc: 99.67%
         | Pred true | Pred false |
GT true  | 2990      | 10         |
GT false | 0         | 0          |


=== Common fake images ===
Acc: 23.40%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 1149      | 351        |


=== Uncommon fake images ===
Acc: 49.80%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 753       | 747        |


In [10]:
# csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_label.csv"
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_Flickr_SDXL_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

# question = "Is this photo real?"
question = "Is this photo [*]?"
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes")
instruct.PrintResult(three_class=True)

  0%|          | 0/750 [00:00<?, ?it/s]

100%|██████████| 750/750 [09:07<00:00,  1.37it/s]

Question: Is this photo [*]?

Acc: 95.23%
         | Pred true | Pred false |
GT true  | 2862      | 138        |
GT false | 148       | 2852       |
Question: Is this photo [*]?

=== Overall ===
Acc: 95.23%
         | Pred true | Pred false |
GT true  | 2862      | 138        |
GT false | 148       | 2852       |


=== Real images ===
Acc: 95.40%
         | Pred true | Pred false |
GT true  | 2862      | 138        |
GT false | 0         | 0          |


=== Common fake images ===
Acc: 93.20%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 102       | 1398       |


=== Uncommon fake images ===
Acc: 96.93%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 46        | 1454       |


In [22]:
# csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_label.csv"
# csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_COCO_SDXL_label.csv"
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_Flickr_SD2_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

question = "Is this photo real?"
# question = "Is this photo [*]?"
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes")
instruct.PrintResult(three_class=True)

100%|██████████| 750/750 [08:18<00:00,  1.50it/s]

Question: Is this photo real?

Acc: 79.92%
         | Pred true | Pred false |
GT true  | 2990      | 10         |
GT false | 1195      | 1805       |
Acc: 79.92%
         | Pred true | Pred false |
GT true  | 2990      | 10         |
GT false | 1195      | 1805       |
Question: Is this photo real?

=== Overall ===
Acc: 79.92%
         | Pred true | Pred false |
GT true  | 2990      | 10         |
GT false | 1195      | 1805       |


=== Real images ===
Acc: 99.67%
         | Pred true | Pred false |
GT true  | 2990      | 10         |
GT false | 0         | 0          |


=== Common fake images ===
Acc: 43.07%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 854       | 646        |


=== Uncommon fake images ===
Acc: 77.27%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 341       | 1159       |


In [24]:
# csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_label.csv"
# csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_COCO_SDXL_label.csv"
csv_path = "/eva_data0/denny/textual_inversion/60k_6k_6k/test_Flickr_SD2_label.csv"
instruct.LoadData_batch(csv_path=csv_path)

# question = "Is this photo real?"
question = "Is this photo [*]?"
acc, confusion_mat, ans_list, labels, label_3class = instruct.QueryImgs_batch(question=question, true_string="yes")

100%|██████████| 750/750 [09:22<00:00,  1.33it/s]


Question: Is this photo [*]?

Acc: 96.60%
         | Pred true | Pred false |
GT true  | 2862      | 138        |
GT false | 66        | 2934       |
Acc: 96.60%
         | Pred true | Pred false |
GT true  | 2862      | 138        |
GT false | 66        | 2934       |


In [25]:
instruct.PrintResult(three_class=True)

Question: Is this photo [*]?

=== Overall ===
Acc: 96.60%
         | Pred true | Pred false |
GT true  | 2862      | 138        |
GT false | 66        | 2934       |


=== Real images ===
Acc: 95.40%
         | Pred true | Pred false |
GT true  | 2862      | 138        |
GT false | 0         | 0          |


=== Common fake images ===
Acc: 96.80%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 48        | 1452       |


=== Uncommon fake images ===
Acc: 98.80%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 18        | 1482       |


In [8]:
#image = Image.open("/eva_data0/denny/coco2014/val2014/COCO_val2014_000000000042.jpg")
image = Image.open("/eva_data0/denny/sd2/coco2014_train/samples/00000.png")
questions = ["Is this photo real?"]
ans = instruct.Query(image, questions)
print(ans)

yes


In [10]:
real_dir = "/eva_data0/denny/coco2014/train2014"
fake_common_dir = "/eva_data0/denny/SemanticError/SD2/0_real/"
fake_uncommon_dir = "/eva_data0/denny/SemanticError/SD2/1_fake/"
real_num = 200
num = [real_num, real_num//2, real_num//2]
#num = [50, 25, 25]
instruct.LoadData3Class(real_dir, fake_common_dir, fake_uncommon_dir, num)

Loading imgs from /eva_data0/denny/coco2014/train2014: 100%|██████████| 200/200 [00:03<00:00, 51.07it/s]
Loading imgs from /eva_data0/denny/SemanticError/SD2/0_real/: 100%|██████████| 100/100 [00:02<00:00, 33.49it/s]
Loading imgs from /eva_data0/denny/SemanticError/SD2/1_fake/: 100%|██████████| 100/100 [00:03<00:00, 32.72it/s]


In [11]:
acc1, mat1, ans1 = instruct.QueryImgs("Is this photo [*]?", true_string="yes")

Answering: 100%|██████████| 400/400 [02:17<00:00,  2.90it/s]

Question: Is this photo [*]?

Acc: 96.25%
         | Pred true | Pred false |
GT true  | 186       | 14         |
GT false | 1         | 199        |


In [12]:
instruct.PrintResult(three_class=True)

Question: Is this photo [*]?

=== Overall ===
Acc: 96.25%
         | Pred true | Pred false |
GT true  | 186       | 14         |
GT false | 1         | 199        |


=== Real images ===
Acc: 93.00%
         | Pred true | Pred false |
GT true  | 186       | 14         |
GT false | 0         | 0          |


=== Common fake images ===
Acc: 99.00%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 1         | 99         |


=== Uncommon fake images ===
Acc: 100.00%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 0         | 100        |


In [15]:
acc1, mat1, ans1 = instruct.QueryImgs("Is this photo common in real world?", true_string="yes")

Answering: 100%|██████████| 2000/2000 [09:48<00:00,  3.40it/s]

Question: Is this photo common in real world?

Acc: 79.40%
         | Pred true | Pred false |
GT true  | 819       | 181        |
GT false | 231       | 769        |


In [16]:
instruct.PrintResult(three_class=True)

Question: Is this photo common in real world?

=== Overall ===
Acc: 79.40%
         | Pred true | Pred false |
GT true  | 819       | 181        |
GT false | 231       | 769        |


=== Common fake images ===
Acc: 64.00%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 180       | 320        |


=== Uncommon fake images ===
Acc: 89.80%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 51        | 449        |


In [25]:
import numpy as np
mat = np.array([[819,181],[231,769]])
print(f'=== Real images ===')
print(f'Acc: {(mat[0,0]/1000)*100:.2f}%')
instruct.PrintConfusion(mat)

=== Real images ===
Acc: 81.90%
         | Pred true | Pred false |
GT true  | 819       | 181        |
GT false | 231       | 769        |


In [17]:
question = "Is this photo generated by a model?"
acc2, mat2, ans2 = instruct.QueryImgs(question, true_string="no")

Answering: 100%|██████████| 2000/2000 [15:43<00:00,  2.12it/s]

Question: Is this photo generated by a model?

Acc: 66.10%
         | Pred true | Pred false |
GT true  | 658       | 342        |
GT false | 336       | 664        |


In [18]:
instruct.PrintResult(three_class=True)

Question: Is this photo generated by a model?

=== Overall ===
Acc: 66.10%
         | Pred true | Pred false |
GT true  | 658       | 342        |
GT false | 336       | 664        |


=== Common fake images ===
Acc: 71.80%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 141       | 359        |


=== Uncommon fake images ===
Acc: 61.00%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 195       | 305        |


In [19]:
acc, confusion_mat, ans = instruct.MultipleAns(ans1, ans2)

Accuracy: 72.85%
         | Pred true | Pred false |
GT true  | 560       | 440        |
GT false | 103       | 897        |


In [20]:
instruct.PrintResult(three_class=True)

Question: Is this photo generated by a model?

=== Overall ===
Acc: 72.85%
         | Pred true | Pred false |
GT true  | 560       | 440        |
GT false | 103       | 897        |


=== Common fake images ===
Acc: 85.80%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 71        | 429        |


=== Uncommon fake images ===
Acc: 93.60%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 32        | 468        |


In [75]:
question = "Is this photo generated by AI?"
instruct.QueryImgs(question, true_string="no")
instruct.PrintResult(three_class=True)

Answering: 100%|██████████| 2000/2000 [10:16<00:00,  3.25it/s]

Question: Is this photo generated by AI?

Acc: 50.75%
         | Pred true | Pred false |
GT true  | 998       | 2          |
GT false | 983       | 17         |
Question: Is this photo generated by AI?

=== Overall ===
Acc: 50.75%
         | Pred true | Pred false |
GT true  | 998       | 2          |
GT false | 983       | 17         |


=== Common fake images ===
Acc: 2.80%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 486       | 14         |


=== Uncommon fake images ===
Acc: 0.60%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 497       | 3          |


In [76]:
instruct.PrintResult(three_class=True)

Question: Is this photo generated by AI?

=== Overall ===
Acc: 50.75%
         | Pred true | Pred false |
GT true  | 998       | 2          |
GT false | 983       | 17         |


=== Common fake images ===
Acc: 2.80%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 486       | 14         |


=== Uncommon fake images ===
Acc: 0.60%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 497       | 3          |


In [16]:
question = "Is this photo common in real world?"
acc1, mat1, ans1 = instruct.QueryImgs(question, true_string="yes")
instruct.PrintResult(three_class=True)

Answering: 100%|██████████| 200/200 [01:25<00:00,  2.35it/s]

Question: Is this photo common in real world?

Acc: 81.00%
         | Pred true | Pred false |
GT true  | 82        | 18         |
GT false | 20        | 80         |
Question: Is this photo common in real world?

=== Overall ===
Acc: 81.00%
         | Pred true | Pred false |
GT true  | 82        | 18         |
GT false | 20        | 80         |


=== Common fake images ===
Acc: 74.00%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 13        | 37         |


=== Uncommon fake images ===
Acc: 86.00%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 7         | 43         |


In [79]:
instruct.PrintResult(three_class=True)

Question: Is this photo common in real world?

=== Overall ===
Acc: 80.25%
         | Pred true | Pred false |
GT true  | 836       | 164        |
GT false | 231       | 769        |


=== Common fake images ===
Acc: 64.00%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 180       | 320        |


=== Uncommon fake images ===
Acc: 89.80%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 51        | 449        |


In [81]:
question = "Is this photo possible in real world?"
instruct.QueryImgs(question, true_string="yes")
instruct.PrintResult(three_class=True)

Answering: 100%|██████████| 2000/2000 [16:25<00:00,  2.03it/s]

Question: Is this photo possible in real world?

Acc: 57.45%
         | Pred true | Pred false |
GT true  | 740       | 260        |
GT false | 591       | 409        |
Question: Is this photo possible in real world?

=== Overall ===
Acc: 57.45%
         | Pred true | Pred false |
GT true  | 740       | 260        |
GT false | 591       | 409        |


=== Common fake images ===
Acc: 39.80%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 301       | 199        |


=== Uncommon fake images ===
Acc: 42.00%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 290       | 210        |


In [82]:
instruct.PrintResult(three_class=True)

Question: Is this photo possible in real world?

=== Overall ===
Acc: 57.45%
         | Pred true | Pred false |
GT true  | 740       | 260        |
GT false | 591       | 409        |


=== Common fake images ===
Acc: 39.80%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 301       | 199        |


=== Uncommon fake images ===
Acc: 42.00%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 290       | 210        |


In [45]:
real_dir = "/eva_data0/denny/coco2014/train2014"
fake_common_dir = "/eva_data0/denny/SemanticError/IF/0_real/"
fake_uncommon_dir = "/eva_data0/denny/SemanticError/IF/1_fake/"
real_num = 1000
num = [real_num, real_num//2, real_num//2]
#num = [50, 25, 25]
instruct.LoadData3Class(real_dir, fake_common_dir, fake_uncommon_dir, num)

Loading imgs from /eva_data0/denny/coco2014/train2014: 100%|██████████| 1000/1000 [00:03<00:00, 252.17it/s]
Loading imgs from /eva_data0/denny/SemanticError/IF/0_real/: 100%|██████████| 500/500 [00:13<00:00, 35.73it/s]
Loading imgs from /eva_data0/denny/SemanticError/IF/1_fake/: 100%|██████████| 500/500 [00:14<00:00, 33.47it/s]


In [49]:
question = "Is this photo common in real world?"
acc1, mat1, ans1 = instruct.QueryImgs(question, true_string="yes")
instruct.PrintResult(three_class=True)

Answering: 100%|██████████| 2000/2000 [09:40<00:00,  3.44it/s]

Question: Is this photo common in real world?

Acc: 74.95%
         | Pred true | Pred false |
GT true  | 836       | 164        |
GT false | 337       | 663        |
Question: Is this photo common in real world?

=== Overall ===
Acc: 74.95%
         | Pred true | Pred false |
GT true  | 836       | 164        |
GT false | 337       | 663        |


=== Common fake images ===
Acc: 39.60%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 302       | 198        |


=== Uncommon fake images ===
Acc: 93.00%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 35        | 465        |


In [50]:
instruct.PrintResult(three_class=True)

Question: Is this photo common in real world?

=== Overall ===
Acc: 74.95%
         | Pred true | Pred false |
GT true  | 836       | 164        |
GT false | 337       | 663        |


=== Common fake images ===
Acc: 39.60%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 302       | 198        |


=== Uncommon fake images ===
Acc: 93.00%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 35        | 465        |


In [51]:
question = "Is this photo generated by a model?"
acc2, mat2, ans2 = instruct.QueryImgs(question, true_string="no")
instruct.PrintResult(three_class=True)

Answering: 100%|██████████| 2000/2000 [17:05<00:00,  1.95it/s]

Question: Is this photo generated by a model?

Acc: 63.95%
         | Pred true | Pred false |
GT true  | 645       | 355        |
GT false | 366       | 634        |
Question: Is this photo generated by a model?

=== Overall ===
Acc: 63.95%
         | Pred true | Pred false |
GT true  | 645       | 355        |
GT false | 366       | 634        |


=== Common fake images ===
Acc: 63.00%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 185       | 315        |


=== Uncommon fake images ===
Acc: 63.80%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 181       | 319        |


In [52]:
instruct.PrintResult(three_class=True)

Question: Is this photo generated by a model?

=== Overall ===
Acc: 63.95%
         | Pred true | Pred false |
GT true  | 645       | 355        |
GT false | 366       | 634        |


=== Common fake images ===
Acc: 63.00%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 185       | 315        |


=== Uncommon fake images ===
Acc: 63.80%
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 181       | 319        |


In [53]:
acc, confusion_mat, ans = instruct.MultipleAns(ans1, ans2)

Accuracy: 69.60%
         | Pred true | Pred false |
GT true  | 554       | 446        |
GT false | 162       | 838        |


In [4]:
# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_t5_instruct", model_type="flant5xl", is_eval=True, device=device)
#model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_t5_instruct", model_type="flant5xxl", is_eval=True, device=device)
model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_vicuna_instruct", model_type="vicuna7b", is_eval=True, device=device)
#model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_vicuna_instruct", model_type="vicuna13b", is_eval=True, device=device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.96s/it]


In [13]:
# For query lots of images
real_dir = "/eva_data0/denny/coco2014/train2014"
fake_dir = "/eva_data0/denny/SemanticError/SD2/0_real/"
#imgs, labels = PrepareData("/eva_data0/denny/SemanticError/SD2/test/")
imgs, labels = PrepareData(real_dir, fake_dir, num=1000)

Loading imgs from /eva_data0/denny/coco2014/train2014: 100%|██████████| 1000/1000 [00:03<00:00, 257.93it/s]
Loading imgs from /eva_data0/denny/SemanticError/SD2/0_real/: 100%|██████████| 1000/1000 [00:22<00:00, 45.26it/s]


In [14]:
# For query lots of images
question = "Is this photo possible in real world?"
acc, confusion_mat, com_acc, com_conf_mat, uncom_acc, uncom_conf_mat = QueryImgs(imgs, labels, question, model, vis_processors, txt_processors, device)

Answering: 100%|██████████| 2000/2000 [17:37<00:00,  1.89it/s]

Question: Is this photo possible in real world?
Acc: 56.85%
         | Pred true | Pred false |
GT true  | 716       | 284        |
GT false | 579       | 421        |


In [ ]:
print(acc)
PrintConfusion(confusion_mat)

print(com_acc)
PrintConfusion(com_conf_mat)
print(uncom_acc)
PrintConfusion(uncom_conf_mat)

In [35]:
# For query lots of images
real_dir = "/eva_data0/denny/coco2014/train2014"
fake_dir = "/eva_data0/denny/SemanticError/SD2/1_fake/"
#imgs, labels = PrepareData("/eva_data0/denny/SemanticError/SD2/test/")
imgs, labels = PrepareData(real_dir, fake_dir, num=200)

Loading imgs from /eva_data0/denny/coco2014/train2014: 100%|██████████| 200/200 [00:01<00:00, 199.41it/s]
Loading imgs from /eva_data0/denny/SemanticError/SD2/1_fake/: 100%|██████████| 200/200 [00:03<00:00, 56.16it/s]


In [33]:
# For query lots of images
question = "Is this photo possible in real world?"
acc, confusion_mat, com_acc, com_conf_mat, uncom_acc, uncom_conf_mat = QueryImgs(imgs, labels, question, model, vis_processors, txt_processors, device)

Answering: 100%|██████████| 200/200 [01:31<00:00,  2.19it/s]

Question: Is this photo possible in real world?
Acc: 60.50%
         | Pred true | Pred false |
GT true  | 74        | 26         |
GT false | 53        | 47         |


In [34]:
print(acc)
PrintConfusion(confusion_mat)

print(com_acc)
PrintConfusion(com_conf_mat)
print(uncom_acc)
PrintConfusion(uncom_conf_mat)

0.605
         | Pred true | Pred false |
GT true  | 74        | 26         |
GT false | 53        | 47         |
0.46
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 27        | 23         |
0.46
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 27        | 23         |


In [18]:
# For query lots of images
question = "Is this photo common in real world?"
acc, confusion_mat = QueryImgs(imgs, labels, question, model, vis_processors, txt_processors, device)

Answering: 100%|██████████| 2000/2000 [10:51<00:00,  3.07it/s]

Question: Is this photo common in real world?
Acc: 83.70%
         | Pred true | Pred false |
GT true  | 812       | 188        |
GT false | 138       | 862        |


In [24]:
question = "Is this photo generated by a model?"
acc, confusion_mat = QueryImgs(imgs, labels, question, model, vis_processors, txt_processors, device, true_string="no")

Answering: 100%|██████████| 2000/2000 [15:58<00:00,  2.09it/s]

Question: Is this photo generated by a model?
Acc: 62.55%
         | Pred true | Pred false |
GT true  | 630       | 370        |
GT false | 379       | 621        |


In [39]:
# For query lots of images
real_dir = "/eva_data0/denny/coco2014/train2014"
fake_dir = "/eva_data0/denny/SemanticError/SD2/real_and_fake/"
#imgs, labels = PrepareData("/eva_data0/denny/SemanticError/SD2/test/")
imgs, labels = PrepareData(real_dir, fake_dir, num=1000)

Loading imgs from /eva_data0/denny/coco2014/train2014: 100%|██████████| 1000/1000 [00:04<00:00, 244.46it/s]
Loading imgs from /eva_data0/denny/SemanticError/SD2/real_and_fake/: 100%|██████████| 1000/1000 [00:10<00:00, 91.52it/s]


In [40]:
question = "Is this photo generated by a model?"
acc, confusion_mat, com_acc, com_conf_mat, uncom_acc, uncom_conf_mat = QueryImgs(imgs, labels, question, model, vis_processors, txt_processors, device, true_string="no")
print(acc)
PrintConfusion(confusion_mat)

Answering:  84%|████████▎ | 1671/2000 [13:03<02:13,  2.46it/s]

In [ ]:
print(acc)
PrintConfusion(confusion_mat)

print(com_acc)
PrintConfusion(com_conf_mat)
print(uncom_acc)
PrintConfusion(uncom_conf_mat)

0.715
         | Pred true | Pred false |
GT true  | 64        | 36         |
GT false | 21        | 79         |
0.78
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 11        | 39         |
0.78
         | Pred true | Pred false |
GT true  | 0         | 0          |
GT false | 11        | 39         |


#### Load an example image

In [ ]:
#raw_image = Image.open("../docs/_static/merlion.png").convert("RGB")
raw_image = Image.open("/eva_data/denny/SemanticError/SD2/1_fake/00000.png").convert("RGB")
#display(raw_image.resize((596, 437)))
display(raw_image)

In [ ]:
# setup device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Load instructBLIP model

In [ ]:
model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_t5_instruct", model_type="flant5xl", is_eval=True, device=device)
#model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_t5_instruct", model_type="flant5xxl", is_eval=True, device=device)
#model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_vicuna_instruct", model_type="vicuna7b", is_eval=True, device=device)
#model, vis_processors, txt_processors = load_model_and_preprocess(name="blip2_vicuna_instruct", model_type="vicuna13b", is_eval=True, device=device)

In [ ]:
vis_processors.keys()

In [ ]:
txt_processors.keys()

In [ ]:
#question = "Which city is this photo taken?"
#question = "What is the animal in the photo?"
question = "Is this photo possible in real world?"

In [ ]:
# use "eval" processors for inference
image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
question = txt_processors["eval"](question)

samples = {"image": image, "text_input": question}

#### generative question answering

In [ ]:
model.predict_answers(samples=samples, inference_method="generate")

#### ranking-based question answering

In [ ]:
# rank answer candidates by their likelihood and return the best answer
answer_candidates = ["Singapore", "London", "Palo Alto", "Tokyo"]

model.predict_answers(samples, answer_list=answer_candidates, inference_method="rank")

#### Ask questions in batch

In [ ]:
batch_size = 3

# create a batch of samples, could be multiple images or copies of the same image
image_batch = image.repeat(batch_size, 1, 1, 1)

# create a batch of questions, make sure the number of questions matches the number of images
question_1 = txt_processors["eval"]("Which city is this photo taken?")
question_2 = txt_processors["eval"]("What time is this during the day?")
question_3 = txt_processors["eval"]("Is it Singapore or London?")

question_batch = [question_1, question_2, question_3]

model.predict_answers(samples={"image": image_batch, "text_input": question_batch}, inference_method="generate")
